<a href="https://colab.research.google.com/github/Looma1116/Kaggle-Study_Seungjoon_Lee/blob/main/w10_regression_cabbage_price_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m venv tutorial-env

Error: Command '['/content/tutorial-env/bin/python3', '-Im', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.


In [ ]:

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
  Using cached kaggle-1.5.6-py3-none-any.whl


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
! kaggle -v

Kaggle API 1.5.6


In [ ]:
!kaggle competitions download -c regression-cabbage-price

  0% 0.00/29.7k [00:00<?, ?B/s]
100% 29.7k/29.7k [00:00<00:00, 21.8MB/s]


In [ ]:
!unzip regression-cabbage-price.zip

Archive:  regression-cabbage-price.zip
  inflating: submit_sample.csv       
  inflating: test_cabbage_price.csv  
  inflating: train_cabbage_price.csv  


In [ ]:
import numpy as np
import torch
xy_train = np.loadtxt('train_cabbage_price.csv', delimiter=',',dtype=np.float32, skiprows=1, usecols=range(1,6))
x_data = torch.from_numpy(xy_train[:,0:-1])
y_data = torch.from_numpy(xy_train[:,[-1]])

xy_test = np.loadtxt('test_cabbage_price.csv', delimiter=',',dtype=np.float32, skiprows=1, usecols=range(1,5))
x_test = torch.from_numpy(xy_test)

In [ ]:
print(x_data)
print(y_data)
print(x_test)

tensor([[ -4.9000, -11.0000,   0.9000,   0.0000],
        [ -3.1000,  -5.5000,   5.5000,   0.8000],
        [ -2.9000,  -6.9000,   1.4000,   0.0000],
        ...,
        [  2.9000,  -2.1000,   8.0000,   0.0000],
        [  2.9000,  -1.6000,   7.1000,   0.6000],
        [  2.1000,  -2.0000,   5.8000,   0.4000]])
tensor([[2123.],
        [2123.],
        [2123.],
        ...,
        [2901.],
        [2901.],
        [2901.]])
tensor([[13.5000,  8.0000, 19.2000,  2.6000],
        [ 1.1000, -3.0000,  9.5000,  0.8000],
        [12.0000,  5.1000, 19.6000,  0.0000],
        [16.6000, 11.9000, 22.5000,  0.1000],
        [22.4000, 17.8000, 27.4000,  0.5000],
        [18.8000, 13.9000, 24.4000,  0.0000]])


In [90]:
epochs = 10001
lr = 0.0001

W = torch.zeros(4,1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

def model(x):
  return x @ W + b

def cost_fn(pred, label):
  diff = pred - label
  return torch.mean(diff**2)

for epoch in range(epochs):
  pred = model(x_data)
  cost = cost_fn(pred,y_data)
  cost.backward()

  if epoch%200 == 0:
    print('Epoch : {:5d}, Loss : {}'.format(epoch, cost.item()))
  
  with torch.no_grad():
    W -= W.grad * lr
    b -= b.grad *lr
    W.grad.zero_()
    b.grad.zero_()

Epoch :     0, Loss : 12610011.0
Epoch :   200, Loss : 2646556.5
Epoch :   400, Loss : 2334200.0
Epoch :   600, Loss : 2284404.25
Epoch :   800, Loss : 2274839.0
Epoch :  1000, Loss : 2271450.75
Epoch :  1200, Loss : 2269020.25
Epoch :  1400, Loss : 2266748.25
Epoch :  1600, Loss : 2264511.0
Epoch :  1800, Loss : 2262290.25
Epoch :  2000, Loss : 2260082.75
Epoch :  2200, Loss : 2257888.75
Epoch :  2400, Loss : 2255707.0
Epoch :  2600, Loss : 2253537.75
Epoch :  2800, Loss : 2251381.5
Epoch :  3000, Loss : 2249237.5
Epoch :  3200, Loss : 2247105.5
Epoch :  3400, Loss : 2244986.0
Epoch :  3600, Loss : 2242878.75
Epoch :  3800, Loss : 2240783.5
Epoch :  4000, Loss : 2238700.5
Epoch :  4200, Loss : 2236629.5
Epoch :  4400, Loss : 2234570.25
Epoch :  4600, Loss : 2232523.0
Epoch :  4800, Loss : 2230487.0
Epoch :  5000, Loss : 2228463.0
Epoch :  5200, Loss : 2226450.5
Epoch :  5400, Loss : 2224449.5
Epoch :  5600, Loss : 2222459.75
Epoch :  5800, Loss : 2220481.5
Epoch :  6000, Loss : 221851

In [93]:
prediction = model(x_test)
print(x_test)
print(prediction)

tensor([[13.5000,  8.0000, 19.2000,  2.6000],
        [ 1.1000, -3.0000,  9.5000,  0.8000],
        [12.0000,  5.1000, 19.6000,  0.0000],
        [16.6000, 11.9000, 22.5000,  0.1000],
        [22.4000, 17.8000, 27.4000,  0.5000],
        [18.8000, 13.9000, 24.4000,  0.0000]])
tensor([[3327.4043],
        [3257.1592],
        [3883.7512],
        [3295.0093],
        [3294.4785],
        [3332.1597]], grad_fn=<AddBackward0>)


In [94]:
import pandas as pd

submit = pd.read_csv('submit_sample.csv')

,Id,Expected
0,0,1000
1,1,1000
2,2,1000
3,3,1000
4,4,1000
5,5,1000


In [96]:
for i in range(len(prediction)) :
  submit['Expected'][i] = prediction[i].item()

print(submit)

   Id  Expected
0   0      3327
1   1      3257
2   2      3883
3   3      3295
4   4      3294
5   5      3332


In [98]:
submit.to_csv('submit.csv', mode='w', index=False)
!kaggle competitions submit -c regression-cabbage-price -f submit.csv -m "first_submit"

100% 54.0/54.0 [00:00<00:00, 248B/s]
Successfully submitted to Regression : Cabbage price